In [1]:
import cv2

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from PIL import Image

In [3]:
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa

In [4]:
from tqdm import tqdm

In [5]:
from sklearn.utils import class_weight, shuffle

In [6]:
import warnings
warnings.filterwarnings("ignore")
WINDOW_SIZE = 299
IMAGE_SIZE  = 512
IMAGE_CHANNELS=3

In [7]:
# Load dataset info
path_to_train = '../input/train/'
data = pd.read_csv('../input/train.csv')

train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
        np.array(image_red_ch), 
        np.array(image_green_ch), 
        np.array(image_blue_ch)), -1)
        #image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug



In [10]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Reshape, Dense, Concatenate, GlobalMaxPooling2D
from keras.layers import BatchNormalization, Input, Conv2D, Lambda, Average
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model

    
def create_model(n_out):
    input_shape=(WINDOW_SIZE,WINDOW_SIZE, IMAGE_CHANNELS)
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
    def slice_images(img):
        for i in range(3):
            for j in range(3):
                x_start=i*WINDOW_SIZE
                x_end=min(x_start+WINDOW_SIZE, IMAGE_SIZE)
                x_start=x_end-WINDOW_SIZE

                y_start=j*WINDOW_SIZE
                y_end=min(y_start+WINDOW_SIZE, IMAGE_SIZE)
                y_start=y_end-WINDOW_SIZE
                
                print("x:{xs},{xe},{xd} y:{ys},{ye},{yd}".format(
                    xs=x_start, xe=x_end, xd=x_end-x_start,
                    ys=y_start, ye=y_end, yd=y_end-y_start
                ))

                #      batch,  x,                y,            channels
                #imgslice = img[:, x_start:x_end, y_start:y_end, :]
                yield Lambda(lambda x: x[:, x_start:x_end, y_start:y_end, :])(img)

    base_model = InceptionV3(include_top=False,
                             weights='imagenet',
                             input_shape=input_shape
                             #input_shape=(WINDOW_SIZE, WINDOW_SIZE, IMAGE_CHANNELS)
                            )
    bn = BatchNormalization()
    conv2d = Conv2D(32, kernel_size=(1,1), activation='relu')
    flt = Reshape((-1,))
    drp = Dropout(0.5)
    def create_branch(in_t):
        print(in_t)
        bn_b = bn(in_t)
        x = base_model(bn_b)
        x = conv2d(x)
        x = Reshape((np.prod(x.shape.as_list()[1:]),))(x)
        x = drp(x)
        return x
    x = Average()([create_branch(i) for i in slice_images(input_tensor)])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model



In [11]:
# warm up model
model = create_model(n_out=28)


x:0,299,299 y:0,299,299
Tensor("lambda_1/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:0,299,299 y:213,512,299
Tensor("lambda_2/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:0,299,299 y:213,512,299
Tensor("lambda_3/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:0,299,299
Tensor("lambda_4/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:213,512,299
Tensor("lambda_5/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:213,512,299
Tensor("lambda_6/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:0,299,299
Tensor("lambda_7/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:213,512,299
Tensor("lambda_8/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)
x:213,512,299 y:213,512,299
Tensor("lambda_9/strided_slice:0", shape=(?, 299, 299, 3), dtype=float32)


In [12]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../working/InceptionV3.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)

# create train and valid datagens
train_generator = data_generator.create_train(
    train_dataset_info[train_indexes], batch_size, (WINDOW_SIZE,WINDOW_SIZE,IMAGE_CHANNELS), augument=True)
validation_generator = data_generator.create_train(
    train_dataset_info[valid_indexes], 32, (WINDOW_SIZE,WINDOW_SIZE,IMAGE_CHANNELS), augument=False)


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_4 (

In [19]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))


ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [13]:


for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True

model.compile(
    loss='binary_crossentropy', 
    optimizer=Adam(1e-03),
    metrics=['acc'])
# model.summary()
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

Epoch 1/2
1651/1651 [==============================] - 3125s 2s/step - loss: 0.1838 - acc: 0.9401 - val_loss: 0.1833 - val_acc: 0.9409
Epoch 2/2
1651/1651 [==============================] - 3199s 2s/step - loss: 0.1784 - acc: 0.9415 - val_loss: 0.1857 - val_acc: 0.9398


In [14]:
# train all layers
for layer in model.layers:
    layer.trainable = True
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=1e-4),
            metrics=['accuracy'])
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[16,64,147,147] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/Adam/gradients/zeros_2486}} = Fill[T=DT_FLOAT, _class=["loc:@train...kpropInput"], index_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/Shape_2487, training_1/Adam/gradients/zeros_2486/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_1/mul/_15877}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_168035_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tens

In [ ]:
# Create submit
submit = pd.read_csv('../input/sample_submission.csv')
predicted = []
draw_predict = []
model.load_weights('../working/InceptionV3.h5')
for name in tqdm(submit['Id']):
    path = os.path.join('../input/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
np.save('draw_predict_InceptionV3.npy', score_predict)
submit.to_csv('submit_InceptionV3.csv', index=False)